In [1]:
import pandas as pd
import numpy as np

In [2]:
FIFA = pd.read_csv('fifa21_train.csv')

In [3]:
FIFA.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [4]:
def cleanHightWeightFoot(df):
    #Height
    player_info_part_clean = df.copy()
    def feet2cm (x):
        height = x.split("'")
        return float(height[0])*30.48 + float(height[1][:-1])*2.54  
    player_info_part_clean['height'] = player_info_part_clean['height'].map(feet2cm)

    #Weight
    player_info_part_clean2 = player_info_part_clean.copy()
    def libs2kg (x):
        return float(x[:-3]) * 0.45359237
    player_info_part_clean2['weight'] = player_info_part_clean2['weight'].map(libs2kg)

    # Left and right foot
    player_info_part_clean3 = player_info_part_clean2.copy()
    mapping = {'Right':0,'Left':1}
    player_info_part_clean3 = player_info_part_clean3.replace({'foot': mapping})
    player_info_part_clean3.head()
    
    return player_info_part_clean3

In [5]:
def clean_money(df):
    
    data = df.copy()
    
    def typeConvert(x):
        x = x[1:]
        if 'K' in x:
            x = 1000*float(x[:-1])
        elif "M" in x:
            x = 1000000*float(x[:-1])
        else:
            float(x)
        return str(x)

    data['release clause'] = data['release clause'].map(typeConvert)
    data['release clause'] = data['release clause'].astype(float)

    data['wage'] = data['wage'].map(typeConvert)
    data['wage'] = data['wage'].astype(float)

    data['value'] = data['value'].map(typeConvert)
    data['value'] = data['value'].astype(float)
    
    return data

In [6]:
#This cleans the player_info 
def clean_player(x):
    x = x.drop(['Loan Date End'],axis=1)
    cols = []
    for i in range(len(x.columns)):
        cols.append(x.columns[i].lower())
    x.columns = cols
        
    x.position.fillna(x.bp, inplace=True)
    x.club.fillna(x['team & contract'], inplace=True)
    x = x.drop(['joined','team & contract'],axis=1)
    
    x = cleanHightWeightFoot(x)
    x = clean_money(x)
    
    return x

In [7]:
clean_player(FIFA)

,id,name,age,nationality,club,bp,position,height,weight,foot,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,185.42,76.203518,1,...,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,185.42,78.925072,0,...,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,167.64,64.863709,0,...,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,CM,187.96,81.193034,0,...,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68


In [8]:
#This cleans the skills part of df

def clean_skills(df):
    skills = df.copy()
    #Dropping columns
    skills = skills.drop(['attacking', 'skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis = 1)

    #Dropping NaNs
    skills = skills.dropna()
    skills = skills.reset_index(drop = True)

    #Replacing NaNs for the mean
    skills = skills.fillna(skills.mean())
    skills['composure'] = skills['composure'].round(decimals = 1)
    
    return skills

In [9]:
def cleanPlayerBaseStats(df):
    # Meaning of the columns:
    # W/F - Weak Foot
    # SM - Skill Moves
    # A/W - Attacking Work Rate
    # D/W - Defending Work Rate
    # IR -  Interception rate?
    # PAC - Pace .. player speed
    # SHO - Shoot
    # DRI - Dribling
    # DEF - Defense
    # PHY - Physical
    # Hits - 
    player_base_stats = df.copy()
   
    player_base_stats = player_base_stats.dropna()
    player_base_stats.reset_index(drop = True)

    player_base_stats_clean = player_base_stats.copy()
    removeStars1 = lambda x: x[:-1]
    removeStars2 = lambda x: x[:-2]
    player_base_stats_clean['w/f'] = player_base_stats_clean['w/f'].map(removeStars2)
    player_base_stats_clean['sm'] = player_base_stats_clean['sm'].map(removeStars1)
    player_base_stats_clean['ir'] = player_base_stats_clean['ir'].map(removeStars2)

    # We change names by data for some features (the ones that can be ordered)
    player_base_stats_clean2 = player_base_stats_clean.copy()
    mapping = {'Low':0,'Medium':1,'High':2}
    player_base_stats_clean2 = player_base_stats_clean2.replace({'a/w': mapping})
    player_base_stats_clean2 = player_base_stats_clean2.replace({'d/w': mapping})

    # We convert them to numeric values
    player_base_stats_clean3= player_base_stats_clean2.copy()
    player_base_stats_clean3['w/f'] = player_base_stats_clean3['w/f'].astype(float)
    player_base_stats_clean3['sm'] = player_base_stats_clean3['sm'].astype(float)
    player_base_stats_clean3['ir'] = player_base_stats_clean3['ir'].astype(float)

    def typeConvert(x):
        if 'K' in x:
            x = 1000*float(x[:-1])
        else:
            float(x)
        return str(x) 

    player_base_stats_clean3['hits'] = player_base_stats_clean3['hits'].map(typeConvert)
    player_base_stats_clean3['hits'] = player_base_stats_clean3['hits'].astype(float)

    return player_base_stats_clean3

In [10]:
def clean_positions(df):
    
    df_remaining = df.drop(['ls',  'st',  'rs',  'lw',  'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
              'lm', 'lcm', 'cm', 'rcm',  'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
              'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova'], axis=1)
    
    #Get the relevant data
    df_edit = df[['ls',  'st',  'rs',  'lw',  'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
              'lm', 'lcm', 'cm', 'rcm',  'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
              'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova']]
    
    pos = df_edit.copy()
    #Create splits and add to the df
    pos[['ls','ls_add']] = pos.ls.str.split('+',1).tolist()
    pos[['st','st_add']] = pos.st.str.split('+',1).tolist()
    pos[['rs','rs_add']] = pos.rs.str.split('+',1).tolist()
    pos[['lw','lw_add']] = pos.lw.str.split('+',1).tolist()
    pos[['lf','lf_add']] = pos.lf.str.split('+',1).tolist()
    pos[['cf','cf_add']] = pos.cf.str.split('+',1).tolist()
    pos[['rf','rf_add']] = pos.rf.str.split('+',1).tolist()
    pos[['rw','rw_add']] = pos.rw.str.split('+',1).tolist()
    pos[['lam','lam_add']] = pos.lam.str.split('+',1).tolist()
    pos[['cam','cam_add']] = pos.cam.str.split('+',1).tolist()
    pos[['ram','ram_add']] = pos.ram.str.split('+',1).tolist()
    pos[['lm','lm_add']] = pos.lm.str.split('+',1).tolist()
    pos[['lcm','lcm_add']] = pos.lcm.str.split('+',1).tolist()
    pos[['cm','cm_add']] = pos.cm.str.split('+',1).tolist()
    pos[['rcm','rcm_add']] = pos.rcm.str.split('+',1).tolist()
    pos[['rm','rm_add']] = pos.rm.str.split('+',1).tolist()
    pos[['lwb','lwb_add']] = pos.lwb.str.split('+',1).tolist()
    pos[['ldm','ldm_add']] = pos.ldm.str.split('+',1).tolist()
    pos[['cdm','cdm_add']] = pos.cdm.str.split('+',1).tolist()
    pos[['rwb','rwb_add']] = pos.rwb.str.split('+',1).tolist()
    pos[['lb','lb_add']] = pos.lb.str.split('+',1).tolist()
    pos[['lcb','lcb_add']] = pos.lcb.str.split('+',1).tolist()
    pos[['rcb','rcb_add']] = pos.rcb.str.split('+',1).tolist()
    pos[['rb','rb_add']] = pos.rb.str.split('+',1).tolist()
    pos[['gk','gk_add']] = pos.gk.str.split('+',1).tolist()
    
    
    #Convert all columns to numeric
    for a in pos:
        pos[a] =  pd.to_numeric(pos[a], errors='coerce')
    
    #Select only columns that are relevant in respect to multicollinearity
    pos_spl_final = pos[['st',  'lw', 'cf', 'cam', 'lm','cm', 'lwb', 'cdm',  'lb', 'lcb', 'gk', 'st_add',
                             'lw_add', 'cf_add','cam_add', 'lm_add', 'cm_add', 'lwb_add', 'cdm_add', 'lb_add', 
                             'lcb_add','gk_add','ova']]
    
    df_new = pd.concat([df_remaining,pos_spl_final], axis=1)    
    
    return df_new

In [11]:

#Global Function 
def preprocess(df):
    df_clean = df.copy()
    df_clean = clean_player(df_clean)
    df_clean = clean_skills(df_clean)
    df_clean = cleanPlayerBaseStats(df_clean)
    df_clean = clean_positions(df_clean)
    
    return df_clean

In [12]:
print(FIFA.shape)
preprocess(FIFA)

(11701, 101)


<ipython-input-8-1b256cc9514a>:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skills = skills.fillna(skills.mean())


,id,name,age,nationality,club,bp,position,height,weight,foot,...,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,...,0,1,1,1,1,1,1,1,1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,...,0,1,1,2,2,2,2,2,2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,...,0,0,1,2,2,2,2,2,2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,...,0,2,2,2,2,2,2,2,2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,...,0,2,2,2,2,2,2,2,2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11417,196848,I. Ávila,29,Peru,Melgar FBC,ST,ST,170.18,69.853225,0,...,0,2,2,2,2,2,2,2,2,65
11418,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,185.42,76.203518,1,...,0,2,2,2,2,2,2,2,2,65
11419,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,185.42,78.925072,0,...,0,2,2,2,2,2,2,2,2,70
11420,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,167.64,64.863709,0,...,0,2,2,2,2,2,2,2,2,67


In [13]:
FIFA_preprocessed = preprocess(FIFA)

<ipython-input-8-1b256cc9514a>:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skills = skills.fillna(skills.mean())


In [14]:
pd.set_option("display.max_columns", None)
FIFA_preprocessed.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,value,wage,release clause,contract,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,1,525000.0,4000.0,801000.0,2015 ~ 2020,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,1682,357,4.0,2.0,2,1,1.0,69,51,63,63,51,60,3.0,58,61,62,63,63,63,59,59,58,54,15,1,0,0,1,1,1,1,1,1,1,1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,0,8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,1961,412,3.0,4.0,2,0,2.0,83,75,68,82,33,71,44.0,77,77,77,76,76,68,57,53,53,48,18,0,0,0,1,1,2,2,2,2,2,2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,0,9000000.0,49000.0,15300000.0,2019 ~ 2022,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,1925,404,4.0,4.0,2,1,2.0,80,77,78,86,27,56,73.0,73,80,79,80,79,74,59,56,53,41,12,2,0,0,0,1,2,2,2,2,2,2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,13,275000.0,4000.0,694000.0,2016 ~ 2021,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,1527,329,2.0,2.0,1,1,1.0,57,44,54,57,57,60,7.0,50,51,51,53,53,56,56,58,57,58,14,2,0,0,2,2,2,2,2,2,2,2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,8,725000.0,2000.0,1400000.0,2018 ~ 2021,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,1664,360,2.0,3.0,0,1,1.0,66,44,60,64,60,66,4.0,56,59,59,61,62,63,64,64,63,61,15,2,0,0,2,2,2,2,2,2,2,2,65


# Start of Day 2

In [210]:
#First step is X - y split
#dropped some extra columns because they werent necesssary for analysiis
X = FIFA_preprocessed.drop(['id','name', 'contract', 'position','ova'], axis = 1)
y= FIFA_preprocessed[['ova']]

In [211]:
#First step is Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=80)

In [212]:
numerical_train = X_train.select_dtypes(include=[np.number])
numerical_test = X_test.select_dtypes(include=[np.number])

In [213]:
categorical_train = X_train.select_dtypes(include = [object])
categorical_test = X_test.select_dtypes(include = [object])

In [269]:

# from sklearn.preprocessing import OneHotEncoder
# encoder_nationality = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['nationality']])#fit determines how many unique alues has each categorical column. You can see below.
# print(encoder_nationality.categories_)
# encoded_train_nationality = encoder_nationality.transform(categorical_train[['nationality']]).toarray()
# encoded_test_nationality = encoder_nationality.transform(categorical_test[['nationality']]).toarray()
# nationality_encoded_train = pd.DataFrame(encoded_train_nationality,columns = encoder_nationality.categories_)
# nationality_encoded_test = pd.DataFrame(encoded_test_nationality,columns = encoder_nationality.categories_)

# encoder_club = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['club']])#fit determines how many unique alues has each categorical column. You can see below.
# print(encoder_club.categories_)
# encoded_train_club = encoder_club.transform(categorical_train[['club']]).toarray()
# encoded_test_club = encoder_club.transform(categorical_test[['club']]).toarray()
# club_encoded_train = pd.DataFrame(encoded_train_club,columns = encoder_club.categories_)
# club_encoded_test = pd.DataFrame(encoded_test_club,columns = encoder_club.categories_)

encoder_bp = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['bp']])#fit determines how many unique alues has each categorical column. You can see below.
print(encoder_bp.categories_)
encoded_train_bp = encoder_bp.transform(categorical_train[['bp']]).toarray()
encoded_test_bp = encoder_bp.transform(categorical_test[['bp']]).toarray()
bp_encoded_train = pd.DataFrame(encoded_train_bp,columns = encoder_bp.categories_)
bp_encoded_test = pd.DataFrame(encoded_test_bp,columns = encoder_bp.categories_)

# categorical_encoded_train = pd.concat([nationality_encoded_train,club_encoded_train,bp_encoded_train], axis = 1)
# categorical_encoded_test = pd.concat([nationality_encoded_test,club_encoded_test,bp_encoded_test], axis = 1)

categorical_encoded_train = pd.concat([bp_encoded_train], axis = 1)
categorical_encoded_test = pd.concat([bp_encoded_test], axis = 1)

new_columns = []
for column in categorical_encoded_train.columns:
    new_columns.append(column[0])
categorical_encoded_train.columns = new_columns

new_columns = []
for column in categorical_encoded_test.columns:
    new_columns.append(column[0])
categorical_encoded_test.columns = new_columns


[array(['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB',
       'RB', 'RM', 'RW', 'RWB', 'ST'], dtype=object)]


In [270]:
#fit train set
#transform both train and test set
from sklearn.preprocessing import MinMaxScaler 
X_transformer = MinMaxScaler().fit(numerical_train)

x_normalized_train = X_transformer.transform(numerical_train)
x_normalized_train = pd.DataFrame(x_normalized_train, columns=numerical_train.columns)

x_normalized_test = X_transformer.transform(numerical_test)
x_normalized_test = pd.DataFrame(x_normalized_test, columns=numerical_train.columns)

In [271]:
X_transformed_train = pd.concat([x_normalized_train, categorical_encoded_train], axis = 1)
X_transformed_test = pd.concat([x_normalized_test, categorical_encoded_test], axis = 1)

In [272]:
# 5. Modeling
#       · Apply model.
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_transformed_train, y_train)

In [273]:
FIFA_validate = pd.read_csv('fifa21_validate.csv')

In [274]:
# Model Validation
#       · R2.
#       · MSE.
#       · RMSE.
#       · MAE.
# If we compare with the train set:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
y_pred = reg.predict(X_transformed_train)
y_true = y_train
print("The R2 is: ",r2_score(y_true, y_pred))
print("The MSE is: ",mean_squared_error(y_true, y_pred, squared=True))
print("The RMSE is: ",mean_squared_error(y_true, y_pred, squared=False))
print("The MAE is: ",mean_absolute_error(y_true, y_pred))

The R2 is:  0.9250782523367546
The MSE is:  3.524653063422818
The RMSE is:  1.8774059399668517
The MAE is:  1.4642765009380863


In [275]:
# If we compare with the test set:
y_pred = reg.predict(X_transformed_test)
y_true = y_test
print("The R2 is: ",r2_score(y_true, y_pred))
print("The MSE is: ",mean_squared_error(y_true, y_pred, squared=True))
print("The RMSE is: ",mean_squared_error(y_true, y_pred, squared=False))
print("The MAE is: ",mean_absolute_error(y_true, y_pred))

The R2 is:  0.9208999120208445
The MSE is:  3.6700959919422194
The RMSE is:  1.9157494595959617
The MAE is:  1.4794924724613365
